In [ ]:
%matplotlib inline

# Gradient Sensitivity.

In this example, we illustrate the use of the [GradientSensitivity][gemseo.post.gradient_sensitivity.GradientSensitivity]
post-processing on the Sobieski's SSBJ problem.

The [GradientSensitivity][gemseo.post.gradient_sensitivity.GradientSensitivity] post-processor plots histograms of the objective and
the constraints derivatives.

By default, the sensitivities are calculated either at the optimum, or when the result
is not feasible, at the least-non feasible point. The iteration where the sensitivities
are computed can be modified via the `iteration` setting.

!!! note
        In some cases, the iteration used to compute the sensitivities corresponds to a
        point for which the algorithm did not request the evaluation of the gradients. In
        this case, a `ValueError` is raised by [GradientSensitivity][gemseo.post.gradient_sensitivity.GradientSensitivity].
        To overcome this issue, one can set the `compute_missing_gradients` setting to True.
        This way, GEMSEO will compute the gradients for the iterations where it is lacking.
        This can be done only if the underlying disciplines are available, explaing why
        why, unlike the other post-processing examples, we need to **post-process directly
        from the MDO scenario**.

!!! warning
        Please note that this extra computation may be expensive depending on the
        [OptimizationProblem][gemseo.algos.optimization_problem.OptimizationProblem] defined by the user. Additionally, keep in mind that
        GEMSEO cannot compute missing gradients for an [OptimizationProblem][gemseo.algos.optimization_problem.OptimizationProblem] that was
        imported from an HDF5 file.


## MDO scenario


Let us first create and execute the MDF scenario on the Sobieski's SSBJ problem.



In [ ]:
from __future__ import annotations

from gemseo import create_discipline
from gemseo import create_scenario
from gemseo.formulations.mdf_settings import MDF_Settings
from gemseo.problems.mdo.sobieski.core.design_space import SobieskiDesignSpace
from gemseo.settings.mda import MDAGaussSeidel_Settings
from gemseo.settings.opt import SLSQP_Settings
from gemseo.settings.post import GradientSensitivity_Settings

disciplines = create_discipline([
    "SobieskiPropulsion",
    "SobieskiAerodynamics",
    "SobieskiMission",
    "SobieskiStructure",
])

formulation_settings = MDF_Settings(
    main_mda_settings=MDAGaussSeidel_Settings(
        max_mda_iter=30,
        tolerance=1e-10,
        warm_start=True,
        use_lu_fact=True,
    ),
)

scenario = create_scenario(
    disciplines,
    "y_4",
    design_space=SobieskiDesignSpace(),
    maximize_objective=True,
    formulation_settings_model=formulation_settings,
)

for name in ["g_1", "g_2", "g_3"]:
    scenario.add_constraint(name, constraint_type="ineq")

scenario.execute(SLSQP_Settings(max_iter=20))

## Post-processing

Let us now post-process the scenario by means of the [GradientSensitivity][gemseo.post.gradient_sensitivity.GradientSensitivity].



In [ ]:
scenario.post_process(
    settings_model=GradientSensitivity_Settings(
        compute_missing_gradients=True,
        save=False,
        show=True,
    ),
)